# HR Data Analysis: Intro

The situation set up by the data set is that a company which is active in the Big Data and Data Analysis space is offer in courses to some of its employees. The company is offer paid training to their employees. Fortunately, many employees have signed up for these paid job trainings. However, they have been running into the situation where upon finishing a course they end up switching companies. The company would like to know whether or a candidate is going to jump ship after finishing the course. The provide dataset encapsulates a number of measured factors such as the number of the training being offered, the education level of the candidate, the developement index of the city for which the company is located, ext ... The data was collected with idea that it could posible to predict whether or not a person is going to leave the company right after completing the paid training.


The comprehensive goal of this brief investagation is produce a predictive model which can accurately decern whether or not a candidate is going to jump ship. Moreover, it is the goal of this investation to create human understandible insights. If these decisions were to be integrated into a real company, it is best not to have a "black box" solution. 

![Image of Yaktocat](https://img.etimg.com/thumb/width-640,height-480,imgsize-126438,resizemode-1,msid-70712251/wealth/earn/should-you-change-your-job-heres-how-to-find-out/jon-change-4.jpg)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import plotly.express as px
import plotly.graph_objects as go



from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.tree import export_graphviz
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.inspection import permutation_importance
from sklearn.model_selection import cross_val_score
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import plot_confusion_matrix

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as Pipeline_im


from IPython.display import Image

from subprocess import call

import matplotlib.pyplot as plt


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing the Data

Upon importing the data we can descernably make a couple of observations
 - Data Points are missing 
 - The dataset is inbalanced
     - Mainly in that most of the sampled participants are Men is STEM majors
 - Most features are catagorical and some are of high cardinality
 
 
 This will be taken into consideration upon building our pipeline

In [ ]:
train = pd.read_csv('../input/hr-analytics-job-change-of-data-scientists/aug_train.csv')
print('DataSet Info')
print('=' * 60)
print(train.info())
print('=' * 60)
print('=' * 60)
print('=' * 60)

print('DataSet Stats')
print('=' * 60)
print(train.describe())

print('=' * 60)
print('=' * 60)
print('=' * 60)


print('Missing Values')
print('=' * 60)
print(train.isna().sum())



train.head()


In [ ]:
test = pd.read_csv('../input/hr-analytics-job-change-of-data-scientists/aug_train.csv')
test.head()

# Exploratory Data Analysis

In [ ]:

train.gender.unique()
male = train[(train['gender'] == 'Male') & (train['target'] == 0)]
female = train[(train['gender'] == 'Female')  & (train['target'] == 0)]
other = train[(train['gender'] == 'Other')  & (train['target'] == 0)]


fig = go.Figure()
fig.add_trace(go.Histogram(x=male['training_hours'],
             name="Male"))
fig.add_trace(go.Histogram(x=female['training_hours'],
             name="Female"))
fig.add_trace(go.Histogram(x=other['training_hours'],
             name="Other"))



# Overlay both histograms
fig.update_layout(barmode='overlay',
                 xaxis_title_text='Number of Training Hours',
                 yaxis_title_text='Count',
                 title_text='Distibution of Candidatas Not Looking For Company Change')
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.5)
fig.show()

In [ ]:
male = train[(train['gender'] == 'Male') & (train['target'] == 1)]
female = train[(train['gender'] == 'Female')  & (train['target'] == 1)]
other = train[(train['gender'] == 'Other')  & (train['target'] == 1)]

fig = go.Figure()
fig.add_trace(go.Histogram(x=male['training_hours'],
             name="Male"))
fig.add_trace(go.Histogram(x=female['training_hours'],
             name="Female"))
fig.add_trace(go.Histogram(x=other['training_hours'],
             name="Other"))


# Overlay both histograms
fig.update_layout(barmode='overlay',
                 xaxis_title_text='Number of Training Hours',
                 yaxis_title_text='Count',
                 title_text='Distibution of Candidatas Looking For Company Change')
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.75)
fig.show()

In [ ]:
train['major_discipline'].unique()

df = train['major_discipline'].value_counts()


fig = px.pie(df, values=df, names=df.index,
            title='Employees by Major')

fig.show()

In [ ]:
# Box Plot Comparing
train = train.replace(np.nan, 'Unknown', regex=True)
train['city_development_index'].unique()
fig = px.box(train, x='target',y="city_development_index", 
             color='gender',
             )
fig.update_layout(
            xaxis_title_text='Looking For New Job')
fig.show()

train['city_development_index'].unique()
fig = px.box(train, x='target',y="city_development_index", 
             color='education_level',
             notched=True)
fig.update_layout(
            xaxis_title_text='Looking For New Job')
fig.show()

# Box Plot Comparing
train = train.replace(np.nan, 'Unknown', regex=True)

# analysze how company size affects the likelyhood that they are getting the training
df = train['company_size']
fig = px.box(train, x='target',y="training_hours", 
             color='education_level',
             notched=True)
fig.update_layout(
            xaxis_title_text='Looking For New Job')
fig.show()

In [ ]:
df = train[(train['target'] == 0)]
df = train['relevent_experience'].value_counts()

fig = px.pie(df, values=df, names=df.index,
            title='Not Looking For New Job')

fig.show()

df = train[(train['target'] == 1)]
df = train['relevent_experience'].value_counts()

fig = px.pie(df, values=df, names=df.index,
            title='Looking For New Job')


fig.show()


## EDA - Key Takeaways

As we can see gender plays list to no roll at all to whether a person is a flight risk or not. The distributions for gender across both groups who chose to stay and to those who chose to leave look nearly identical. Another key takeaway is the degree to which our data is imbalanced. Seventy two percent of candidates have relevent experience, eighty nine percent of of candidates have STEM backgrounds, and the overwhelming majority of candidates are male. Therefore, it should be necissary that we take into account this imbalance when defining our pipeline. 

It should also be noted that across both gender and education levels we see that the city's level of developement plays a significant role in determining whether the participant leaves or not. This makes sense as a person living in a city with a low developement index logically is going to be more likely to look elsewhere for another opportunity, Additionally, it is the reason why top companies such as google locate their offices in growing cities with high levels of developement. 

# Data Cleaning and Model Definition

    - Define Pipeline,
        - Impute Missing Values or Drop the Columns
        - One Hot Encode the Low Cardinality Categorical Data
        - Utilize SMOTE to deal with imbalance (Synthetic Minority Oversampling Technique)
        - Utilize Cross Validation to evaluate our model to get a better sense of our accuracy

Resources: \ 
[Cross Validation](https://www.kaggle.com/alexisbcook/cross-validation) \ 
[SMOTE - Over Sampling](https://machinelearningmastery.com/smote-oversampling-for-imbalanced-classification/)

 

In [ ]:
train = pd.read_csv('../input/hr-analytics-job-change-of-data-scientists/aug_train.csv')
test = pd.read_csv('../input/hr-analytics-job-change-of-data-scientists/aug_test.csv')
train.isna().sum()


In [ ]:
# Seperate input data from the target data
y = train['target']
X_full = train.drop('target', axis=1)
X_train_full = X_full.drop('enrollee_id', axis=1)

y_test_full = train['target']
X_test_full = train.drop('target', axis=1)
X_enrollee_id = X_test_full['enrollee_id']
X_test_full = X_test_full.drop('enrollee_id', axis=1)








# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols_low = [cname for cname in X_train_full.columns if
                    X_train_full[cname].nunique() < 10 and 
                    X_train_full[cname].dtype == "object"]

categorical_cols_high = [cname for cname in X_train_full.columns if
                    X_train_full[cname].nunique() > 10 and 
                    X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if 
                X_train_full[cname].dtype in ['int64', 'float64']]

# # Keep selected columns only
my_cols = categorical_cols_low + numerical_cols 

X_test = X_test_full[my_cols].copy()

X = X_full[my_cols].copy()

In [ ]:
print(f'Columns to Include: {my_cols}')
print(f'Columns Getting Excluded {categorical_cols_high}')


Both 'city' and 'experience can be reasoned to be redundant in 'relevent_expericence' and 'city_developement_index', so I'm alright with loosing that bit of information.

In [ ]:
def make_model(model=DecisionTreeClassifier, max_depth=None):
    # Preprocessing for numerical data
    numerical_transformer = SimpleImputer(strategy='constant')

    # Preprocessing for categorical data
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    # Bundle preprocessing for numerical and categorical data
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, numerical_cols),
            ('cat', categorical_transformer, categorical_cols_low)
        ])



    if max_depth:
    # Define model
        model = Pipeline_im([('passthrough', SMOTE()),
           ('model',model(max_depth=max_depth))])
    else:
        model = Pipeline_im([('passthrough', SMOTE()),
           ('model',model())])

    # Bundle preprocessing and modeling code in a pipeline
    clf = Pipeline(steps=[('preprocessor', preprocessor),  
                          ('model', model)
                         ])

    # Cross Validate the training data
    auc = cross_val_score(clf, X, y,
                              cv=5,
                              scoring='roc_auc')
   
    # Preprocessing of training data, actually fit a model 
    clf.fit(X, y)
    
    avg_auc = auc.mean()
    std_auc = auc.std()
    
    print('ROC//AUC Score w/ Cross Validation:', avg_auc)
    print('AUC Score Standard Deviation: ', std_auc)

    return avg_auc, std_auc, clf




In [ ]:
roc_DT_limitted, std_auc_DT, clf_DT_limtted = make_model(DecisionTreeClassifier, max_depth=4)


In [ ]:
# Extract the onehot encoded features out of the Pipeline
feature_names = clf_DT_limtted.named_steps['preprocessor'].transformers_[1][1]\
   .named_steps['onehot'].get_feature_names(categorical_cols_low)
feature_names = numerical_cols + list(feature_names) 



# Export a Visual Representation of the tree
export_graphviz(clf_DT_limtted['model']['model'], out_file='tree.dot', 
                feature_names = feature_names,
                class_names = ['stay', 'leave'],
                rounded = True, proportion = False, 
                precision = 2, filled = True)


call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=600'])
Image(filename = 'tree.png')

In [ ]:
auc_ADA, std_ADA, clf_ADA = make_model(AdaBoostClassifier)


In [ ]:

auc_XGB, std_XGB, clf_XGB = make_model(GradientBoostingClassifier, 4)


In [ ]:
roc_KNN, std_KNN, clf_KNN = make_model(KNeighborsClassifier)

In [ ]:
ax = plt.gca()
ada_disp = plot_roc_curve(clf_ADA, X_test, y_test_full, ax=ax, alpha=0.8, name='AdaBoostClassifier')
XGB_disp = plot_roc_curve(clf_XGB, X_test, y_test_full,  ax=ax, alpha=0.8, name='GradientBoostingClassifier')
KNN_disp = plot_roc_curve(clf_KNN, X_test, y_test_full,  ax=ax, alpha=0.8, name='KNeighborsClassifier')
DT_disp =  plot_roc_curve(clf_DT_limtted, X_test, y_test_full,  ax=ax, alpha=0.8, name='DecisionTree')


In [ ]:
disp = plot_confusion_matrix(clf_KNN, X_test, y_test_full)
disp.ax_.set_title('KNeighborsClassifier')

disp1 = plot_confusion_matrix(clf_XGB, X_test, y_test_full)
disp1.ax_.set_title('XGBoostClassifier')

disp2 = plot_confusion_matrix(clf_DT_limtted, X_test, y_test_full)
disp2.ax_.set_title('Decision Tree')

disp3 = plot_confusion_matrix(clf_ADA, X_test, y_test_full)
disp3.ax_.set_title('AdaBoostClassifier')

In [ ]:
# Permutation Importance 
results = permutation_importance(clf_XGB, X_test, y_test_full,
                                n_repeats = 30,
                                random_state=69)

In [ ]:
results_df = pd.DataFrame(results.importances_mean)
results_df['Features'] = my_cols
results_df = results_df.rename(columns={0 : 'Average Importance'})

fig = px.bar(results_df, x='Features', y='Average Importance')
fig.show()

# Conclusion - Key Take Aways

Since we are getting varying results with our models which at best are imperfectly accurate, one has to question to validity of productionalizing this model into an actual HR department in a company. Analyzing the ROC curve which visualizes True Positivity Rate vs False Positive Rate for every threshold we can give the classifier. True Positivity Rate is that rate of truly positive results that our model predicts over the numeber of actual positive. False Positivity Rate is the number of False Positives detected over the actual number of negatives in the dataset. Our classifiers can be tuned based on a threshold so that it is less likely to either yeild false positives or false negatives, however there is a trade off. The ROC (Reciever Operator Characteristic) Curve demostrates this tradeoff. If we make some broad estimates with interpretting our results, we can see that in the best of conditions if we want to truly identify 80% of true fight risks, we would be falsely acusing 50% of our candidates. Impementing any threshold (any point on the curve) might lead to demotivating effects of the employees and consiquently make them more likely to leave your company. Thus it is my opinion that a productionalized model should not be used over traditional HR resources.

Utilizing our model as a tool to extract insight, we see as was noting in the exploratory data analysis portion of the notebook city developement is by and large the most prodominant determinant. This confirms what we already know. This makes sense as a person living in a city with a low developement index logically is going to be more likely to look elsewhere for another opportunity, Additionally, it is the reason why top companies such as google locate their offices in growing cities with high levels of developement. From the visualized tree we see that if a person lives in a low developement city, they are less likely to leave if they are working in a larger company.

Another important note is that gender play absolutely no role in determining whether or not a candidate is a flight risk. Unfortunately, sexism and bias are prevelent in our society and historically a person might get overlooked for a training based on their gender. Our finding go beyond the moral arguement, to find this empirically nonsensical. 